## Making Petfinder API request and storing API data

### Call in packages

In [2]:
import time
import random
import pandas as pd
import numpy as np
import petpy

import json
import sys
import re

### Pulling data from the petfinder API

In [20]:
# key = XXXX
# secret = XXXX

# pf = petpy.Petfinder(key=key, secret=secret)
# pf._auth

In [12]:
## Function to request data from the petfinder API  
def get_data(city_state, pages=100 ):

    ## using the petpy wrapper to collect all data
    city_df = pf.animals(location=city_state,
                         results_per_page=500,
                         pages=pages,
                         return_df=True)

    ## using the wrapper again to return json data inluding picture URLs
    city_json = pf.animals(location=city_state,
                           results_per_page=100,
                           pages=pages)
    
    
    return city_json, city_df

In [16]:
## Function to merge json and pandas objects, do initial feature extraction
def data_pre_process(dog_data_df, dog_data_jsn, city_initials):
    
    dog_data_json = []
    for dog in dog_data_jsn['animals']:
        dict = {}
        dict['id'] = dog['id']
        dict['num_photo'] = len(dog['photos'])
        if len(dog['photos']) > 0:
            dict['photo'] = dog['photos'][0]['small']
        dog_data_json.append(dict)

    dog_pht_data = pd.DataFrame(dog_data_json)

    ##merging json and dataframe data together
    dog_data = dog_data_df.merge(dog_pht_data, how='inner', on='id')

    ## subseting by animal and status
    dog_data = dog_data[dog_data['animal_type'] == 'dog']
    dog_data = dog_data[dog_data['status'] != 'found']

    dog_data['pub_year'] = dog_data['published_at'].str[:4]
    dog_data['pub_month'] = dog_data['published_at'].str[5:7]
    dog_data['pub_day'] = dog_data['published_at'].str[5:7]

    dog_data['missing_description'] = pd.isnull(dog_data['description'])

    file_name = 'Petfinder_Project/dog_data_' + city_initials + '.csv'

    dog_data.to_csv(file_name, index=False)
    
    return dog_data


#### Pull Bay Area data

In [13]:
# STL_json, STL_df = get_data("Oakland, CA")

In [17]:
dog_data_SF = data_pre_process(SF_df, SF_json, "SF")

In [21]:
dog_data_SF.shape

#### Extracting Seattle data for validation

In [13]:
# STL_json, STL_df = get_data("Seattle, WA")

In [17]:
dog_data_STL = data_pre_process(STL_df, STL_json, "STL")

In [19]:
dog_data_STL.shape

(5961, 49)